# データセット構成状態モニタリング

ここでは、作成いただいたデータセット（実験パッケージ）の構成状態をモニタリングします。以下のセルを実行することで、それぞれのデータセットがその管理品質上、適合的であるかどうかを確認できます。モニタリングの結果はREADME.mdへ自動的に反映されます。

![result_on_README](https://raw.githubusercontent.com/NII-DG/workflow-template/main/images/result_on_README.png)

## ◇◆◇開発メモ◇◆◇

### 処理の流れ

- DMPの情報として、`datasetStructure`が定義されている想定。何種類かのスキームから選んでもらうようなイメージでいる。
    - 現在、設定しているのは「RCOS_with_code」「RCOS_only_data」「YODA」の３種類。
- 実験名のリストとスキーム名のリストとをそれぞれ作成し、モニタリング処理を回す。
    - 現状、スキームごとに望ましいフォルダ構成を定義しており、それぞれのパスが実際に存在するかを確認し、その結果を出力するような実装になっている。


In [ ]:
import os
import json
from typing import List

os.chdir('..')
from util.scripts import utils

def get_ideal_paths(scheme_name: str, path_to_dataset: str) -> List[str]:
    # 存在することが望ましいディレクトリ・ファイルのリスト
    ideal_paths_RCOS_with_code = [
        path_to_dataset + '/source',
        path_to_dataset + '/input_data',
        path_to_dataset + '/output_data',
        path_to_dataset + '/meta_data.json'
    ]

    ideal_paths_RCOS_only_data = [
        path_to_dataset + '/input_data',
        path_to_dataset + '/output_data',
        path_to_dataset + '/meta_data.json'
    ]

    ideal_paths_YODA = [
        path_to_dataset + '/ci',
        path_to_dataset + '/code',
        path_to_dataset + '/code/tests',
        path_to_dataset + '/docs',
        path_to_dataset + '/docs/build',
        path_to_dataset + '/docs/source',
        path_to_dataset + '/envs',
        path_to_dataset + '/inputs',
        path_to_dataset + '/inputs/data',
        path_to_dataset + '/outputs',
        path_to_dataset + '/CHANGELOG.md',
        path_to_dataset + '/HOWTO.md',
        path_to_dataset + '/README.md',
    ]

    return eval('ideal_paths_' + scheme_name)

###########################################################################################################


def monitor_dataset_structure(scheme_name: str, paths_to_dataset: List[str]) -> bool:
    ideal_paths: List[str] = []
    dataset_name: str = ''
    isOK: bool = True

    for dataset in paths_to_dataset:
        ideal_paths = get_ideal_paths(scheme_name, dataset)
        dataset_name = os.path.basename(dataset)

        print('================================ Monitoring Dataset Structure ================================')
        print('Dataset name: ' + dataset_name)
        print(' STATUS    PATH')

        for path in ideal_paths:
            if os.path.exists(path):
                print('  OK    ' + path)
            else:
                print('  NG    ' + path)
                print('        [WARN] Your dataset structure is deprecated.')
                if isOK:
                    isOK = False

        print('\nMonitered by "' + scheme_name + '" scheme.\n')
    return isOK

###########################################################################################################

# スクリプトファイルの拡張子リストを定義（あるいは読み出し）
# something cool...


def main() -> None:
    perspective = 'datasetStructure'
    path_monitoring_params = utils.fetch_monitoring_param_file_path()

    # データセットのパスのリストを作成
    base_path = '/home/jovyan/experiments/'
    files = os.listdir(base_path)
    paths_to_dataset = [base_path + f for f in files if os.path.isdir(os.path.join(base_path, f))]

    # データセット構成のスキームを取得
    scheme_name = ''
    with open(path_monitoring_params, "r") as f:
        dmp = json.load(f)
        scheme_name = dmp["datasetStructure"]

    isOK: bool = monitor_dataset_structure(scheme_name, paths_to_dataset)
    utils.reflect_monitoring_results(perspective, isOK)


if __name__ == '__main__':
    main()


以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

%cd ~/
try:
    pm.execute_notebook(
        'WORKFLOWS/FLOW/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters=dict(SAVE_MESSAGE='モニタリング（データセット構成）', IS_RECURSIVE=False, TO_GIT=True)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)